## 파이콘 벡터 DB에 저장 및 검색이 가능하도록 설정

In [ ]:
# 0. 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain_chroma
# %pip install langchain-pinecone
# %pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고 분할 #######################################
loader = Docx2txtLoader('law_2.docx') # --> 인스턴스 생성

text_splitter = RecursiveCharacterTextSplitter( # --> 분할 설정
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

# 2. 임베딩 -> 백터 데이터베이스에 저장  #########################
## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(
    model='text-embedding-3-large'
)

## 3. Pinecone Vector DB
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore.from_documents(
    index_name='law-index-2',
    embedding=embedding,
    documents=document_list,    
)

# 4. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색  #############
## 4.1 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 4.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

# 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###########
## 5.1 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 5.2 프롬프트 변수에 값 설정(format 함수)
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)
 
## 5.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 5.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message.content

'전세사기피해자에 대한 금융지원은 주로 정부와 금융기관이 협력하여 피해자들이 겪는 재정적 어려움을 완화하고 주거 안정을 도모하기 위한 조치들을 포함하고 있습니다. 전세사기피해자 금융지원의 주요 내용은 다음과 같습니다:\n\n1. **대출 지원**: 피해자들이 기존 전세금 회수가 어려운 상황에서 새로운 주거지를 마련하거나 계속 거주할 수 있도록 저리의 대출을 제공하는 프로그램이 있습니다. 이러한 대출은 전세금 반환 청구 소송 등을 고려하여 장기적으로 상환할 수 있도록 설계됩니다.\n\n2. **이자 지원**: 금융기관에서 제공하는 대출에 대한 이자 지원을 통해 피해자들의 부담을 줄이는 방안이 있습니다. 정부 혹은 지자체에서 이자의 일부를 지원함으로써 피해자들이 저렴한 비용으로 대출을 이용할 수 있도록 돕습니다.\n\n3. **주거 안정 지원**: 피해자들이 새로운 전세 계약을 체결하거나 기존 계약을 유지할 수 있도록 주거 지원금을 제공할 수 있습니다. 또한, 주거 이전 비용이나 임시 주거 시설 제공 등의 지원이 포함될 수 있습니다.\n\n4. **법률 지원 및 상담 서비스**: 전세사기 피해자들에게 법률적인 조언을 제공하고, 필요에 따라 변호사 선임 비용을 지원하는 등 법적 대응을 위한 지원이 제공됩니다. 이를 통해 피해자들이 법적 절차를 원활하게 진행할 수 있도록 돕습니다.\n\n5. **지원 절차 간소화**: 피해자들이 금융 지원을 보다 신속하게 받을 수 있도록 신청 절차를 간소화하고, 관련 서류 준비를 돕는 지원 서비스가 제공됩니다.\n\n이외에도 각 지자체나 개별 금융기관에서 추가적인 지원 프로그램을 운영할 수 있으니, 피해자들은 본인의 상황에 맞는 구체적인 지원책을 알아보시는 것이 좋습니다. 정부의 공식 웹사이트나 피해자 지원 센터에 문의하여 최신 정보와 맞춤형 상담을 받는 것이 도움이 될 것입니다.'